# REVIEW > DF로 로딩 함수

In [51]:
import json
import pandas as pd
from dateutil import parser
from datetime import datetime

# file 로딩 > df 만드는 함수
def load_data(data_path):
    # 파일로딩
    with open(f"{data_path}", "r", encoding="utf-8") as file:
        json_data = json.load(file)

    # 로딩 데이터 > df로 만들기
    df = pd.DataFrame(json_data)
    # print(df.head())
    # 데이터 전처리
    df['date'].str.replace(".","-")

    # date 형태(2025.05.06)가 아닌경우, 날짜로 바꾸기
    # 문자열을 날짜로 변환, 오류가 나는 경우 NaT로 처리
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # 오늘 날짜
    today = pd.Timestamp(datetime.today().date())

    # NaT인 경우 오늘 날짜로 대체
    df['date'] = df['date'].fillna(today)

    # df 리턴하기 
    return df

In [52]:
datetime.today().date()

datetime.date(2025, 6, 5)

In [53]:
# 함수 호출부
data_path = "res/1.gangneung_mare.json"
df = load_data(data_path)
print(len(df))
df.head()

420


C:\Users\Admin\AppData\Local\Temp\ipykernel_25156\2330629591.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


,review,stars,date
0,"바닷가 바로앞이라 너무좋아요.\n객실도 깨끗하고, 샤워기 수압좋고 따뜻한물 잘 나오...",5,2025-06-05
1,오션뷰라 진짜 좋았어요!! ㅎㅎㅎ\n제 생일이라 남자친구하고 처음 오션뷰 예약해서 ...,5,2025-06-05
2,"황금연휴에 방문했는데 오션뷰가 정말 예쁘더라고요👍\n저는 2층에 머물렀는데, 창밖으...",5,2025-05-06
3,위치좋고 직원들 전부 친절하고 재방문의사100%입니다,5,2025-06-05
4,좋았습니다 수고하세요,5,2025-06-05


# mysql DB 연결하기

In [54]:
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine

# .env 파일을 찾아서 환경변수 로딩
load_dotenv('.env', override=True)

# 메모리에 로딩된 환경변수 값을 가져오기
USER = os.environ.get("USER")
PASSWORD = os.environ.get("PASSWORD")
HOST = os.environ.get("HOST")
DB_NAME = os.environ.get("DB_NAME")
PORT = 3306

# DB_RUL = f"mysql+pymsql://{id}:{pw}@{서버ip}:{port}/{db}"
db_url = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}"

# df > mysql 테이블에 저장

In [55]:
# df -> db save
def save_to_db(df, table_name):
    try:
        # SQLAlchemy 엔진 생성
        engine = create_engine(db_url)

        # DataFrame을 MySQL 테이블에 저장
        df.to_sql(
            name = table_name,
            con=engine,
            if_exists='append',  # 기존 데이터에 추가
            index=False,         # 인덱스는 저장하지 않음
            method='multi'       # 성능 향상을 위한 bulk insert
        )

        print(f"SQLAlchemy를 통해 {len(df)}개의 리뷰가 성공적으로 저장되었습니다.")

    except Exception as e:
        print(f"SQLAlchemy 저장 중 오류 발생: {e}")

## 숙소 정보 DB 저장

In [56]:
import pandas as pd
df = pd.read_csv("res/accommodation_info.csv", encoding='utf-8')
# 숙소정보 DB 저장
table_name = 'accommodation_info'

df.head()
save_to_db(df, table_name)
# print(f"{table_name}에 데이터 저장 완료")

SQLAlchemy를 통해 4개의 리뷰가 성공적으로 저장되었습니다.


## 리뷰데이터 저장하기

In [57]:
# 전처리
data_path = "res/1.gangneung_mare.json"
df = load_data(data_path)

# df에 accommodation_id 추가하기
# 강릉 마레는 accommodation_id = 1
df['accommodation_id'] = 1
# df.head()

# DB 테이블에 저장하기
table_name = "accommodation_review"
save_to_db(df, table_name)

# save_to_db(df, "accommodation_review")
# save_to_db(df, table_name)

SQLAlchemy를 통해 420개의 리뷰가 성공적으로 저장되었습니다.


C:\Users\Admin\AppData\Local\Temp\ipykernel_25156\2330629591.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


## 여러개의 파일을 일괄로 전처리하기

- 함수정의

In [58]:
import json
import pandas as pd
from dateutil import parser
from datetime import datetime

# file 로딩 > df 만드는 함수
def load_data_multi(acco_id, data_path):
    # 파일로딩
    with open(f"{data_path}", "r", encoding="utf-8") as file:
        json_data = json.load(file)

    # 로딩 데이터 > df로 만들기
    df = pd.DataFrame(json_data)
    # print(df.head())
    # 데이터 전처리
    df['date'].str.replace(".","-")

    # date 형태(2025.05.06)가 아닌경우, 날짜로 바꾸기
    # 문자열을 날짜로 변환, 오류가 나는 경우 NaT로 처리
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # 오늘 날짜
    today = pd.Timestamp(datetime.today().date())

    # NaT인 경우 오늘 날짜로 대체
    df['date'] = df['date'].fillna(today)
    
    # df 에 accommodation_id 추가하기
    # 강릉 마레는 accommodation_id =1
    df['accommodation_id'] = 1

    # df 리턴하기 
    return df

- 일괄실행

In [59]:
# 전처리하고 저장해야하는 데이터 파일이 10
MAPPING = {
    # 1: './res/1.gangneung_mare.json',
    2: './res/2.insadong_ninetree.json',
    3: './res/3.kyongpo_the_hongc.json',
    # 4: './res/4.hongcheon_vivaldi.json'
}
for acco_id, data_path in MAPPING.items():
# 전처리
    # data_path = "res/1.gangneung_mare.json"
    df = load_data_multi(acco_id, data_path)

    # DB 테이블에 저장하기
    table_name = "accommodation_review"
    save_to_db(df, table_name)


C:\Users\Admin\AppData\Local\Temp\ipykernel_25156\2825726136.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


SQLAlchemy를 통해 400개의 리뷰가 성공적으로 저장되었습니다.
SQLAlchemy를 통해 420개의 리뷰가 성공적으로 저장되었습니다.
